In [35]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import numpy as np
import pandas as pd

In [38]:
# Load the saved model
model_bal = load_model("temp_balikpapan.h5")
model_ber = load_model("temp_berau.h5")
model_bon = load_model("temp_bontang.h5")
model_kut = load_model("temp_kutai.h5")
model_pen = load_model("temp_penajam.h5")
model_sam = load_model("temp_samarinda.h5")

df_bal = pd.read_csv('balikpapan_weather.csv')
df_ber = pd.read_csv('berau_weather.csv')
df_bon = pd.read_csv('bontang_weather.csv')
df_kut = pd.read_csv('kutai_weather.csv')
df_pen = pd.read_csv('penajam_weather.csv')
df_sam = pd.read_csv('samarinda_weather.csv')

In [48]:
def predict_temperature(date, year):

    if city == 'Balikpapan':
      model = model_bal
      df = df_bal
    elif city == 'Berau':
      model = model_ber
      df = df_ber
    elif city == 'Bontang':
      model = model_bon
      df = df_bon
    elif city == 'Kutai Kertanegara':
      model = model_kut
      df = df_kut
    elif city == 'Penajam':
      model = model_pen
      df = df_pen
    elif city == 'Samarinda':
      model = model_sam
      df = df_sam
    else:
      return {
          'message':'City not found'
      }

    year, month, day = map(int, date.split('-'))

    # Tanggal yang akan diprediksi suhunya
    input_date = datetime(year, month, day)

    # Tentukan tanggal referensi
    reference_date = datetime(2015, 1, 1)

    # Hitung perbedaan hari antara tanggal input dan tanggal referensi
    delta_days = (input_date - reference_date).days

    data = df[['Date', 'Temp']]
    data = data.set_index('Date')
    # minmax scaler
    scaler = MinMaxScaler(feature_range = (0, 1))
    data = scaler.fit_transform(data)

    # Ubah perbedaan hari menjadi representasi numerik
    input_date_representation = scaler.transform([[delta_days]])[0]

    time_step = 100  # Set the appropriate value based on your problem and data characteristics

    # Buat input data untuk prediksi
    input_data = []
    for i in range(len(data) - time_step + 1, len(data)):
        input_data.append(data[i, 0])
    input_data.append(input_date_representation[0])

    # Konversi input data menjadi array NumPy
    input_data = np.array(input_data)

    # Reshape input data menjadi bentuk yang dapat diterima oleh model
    input_data = np.reshape(input_data, (1, time_step, 1))

    # Lakukan prediksi suhu
    predicted_temperature = model.predict(input_data)

    # Reshape predicted_temperature to have the expected dimensions
    predicted_temperature = np.reshape(predicted_temperature, (1, 1))

    # Apply inverse_transform to convert the predicted_temperature to the original scale
    predicted_temperature = scaler.inverse_transform(predicted_temperature)[0][0]

    return {
        'temp': predicted_temperature,
        'city': city,
        'date': date
    }

In [49]:

# Contoh pemanggilan fungsi
date = '2023-06-25'
city = 'Balikpapan'

predicted_temp = predict_temperature(date, city)
print(predicted_temp)

1/1 [==============================] - 0s 45ms/step
{'temp': 27.610033, 'city': 'Balikpapan', 'date': '2023-06-25'}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
